In [2]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout,experimental
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import tensorflow as tf

import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

In [3]:
train_path="train"
test_path="test"
val_path="validate"

In [4]:
def loadData():
    
    x_train=[]
    x_test=[]
    x_val=[]
    
    for folder in os.listdir(train_path):

        sub_path=train_path+"/"+folder

        for img in os.listdir(sub_path):

            image_path=sub_path+"/"+img

            img_arr=cv2.imread(image_path)

            img_arr=cv2.resize(img_arr,(224,224))

            x_train.append(img_arr)


    for folder in os.listdir(test_path):

        sub_path=test_path+"/"+folder

        for img in os.listdir(sub_path):

            image_path=sub_path+"/"+img

            img_arr=cv2.imread(image_path)

            img_arr=cv2.resize(img_arr,(224,224))

            x_test.append(img_arr)

    for folder in os.listdir(val_path):

        sub_path=val_path+"/"+folder

        for img in os.listdir(sub_path):

            image_path=sub_path+"/"+img

            img_arr=cv2.imread(image_path)

            img_arr=cv2.resize(img_arr,(224,224))

            x_val.append(img_arr)
            
            
    return x_train, x_test, x_val

In [5]:
# Generate Lable 

def generateLable():
    train_datagen = ImageDataGenerator(rescale = 1./255)
    test_datagen = ImageDataGenerator(rescale = 1./255)
    val_datagen = ImageDataGenerator(rescale = 1./255)
    
    training_set = train_datagen.flow_from_directory("train",
                                                     target_size = (224, 224),
                                                     batch_size = 32,
                                                     class_mode = 'categorical')
    test_set = test_datagen.flow_from_directory(test_path,
                                                target_size = (224, 224),
                                                batch_size = 32,
                                                class_mode = 'sparse')
    val_set = val_datagen.flow_from_directory(val_path,
                                                target_size = (224, 224),
                                                batch_size = 32,
                                                class_mode = 'sparse')
    return training_set.classes, test_set.classes, val_set.classes


In [6]:
x_train, x_test, x_val = loadData()
train_y, test_y, val_y = generateLable()

Found 149 images belonging to 5 classes.
Found 69 images belonging to 5 classes.
Found 67 images belonging to 5 classes.


In [7]:
IMG_SIZE = 224
def augment(image, label):
    img = tf.cast(image, tf.float32)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img = (img / 255.0)
    img = tf.image.random_crop(img, size=[IMG_SIZE, IMG_SIZE, 3])
    img = tf.image.random_brightness(img, max_delta=0.5)
    return img, label  

In [8]:
#Data Augmentation
augmented_images = list(map(augment,x_train,train_y))

In [9]:
for image in augmented_images:
    x_train.append(image[0])
    train_y = np.append(train_y,image[1])

In [10]:
train_x=np.array(x_train)
test_x=np.array(x_test)
val_x=np.array(x_val)

In [11]:
train_x=train_x/255.0
test_x=test_x/255.0
val_x=val_x/255.0

In [12]:
res = ResNet50(input_shape=(224,224,3), weights='imagenet', include_top=False)

In [13]:
for layer in res.layers:
    layer.trainable = False

In [14]:
x = Flatten()(res.output)

In [16]:
#adding output layer.Softmax classifier is used as it is multi-class classification
prediction = Dense(5, activation='softmax')(x)

model = Model(inputs=res.input, outputs=prediction)

In [17]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [18]:
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

In [19]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)
#Early stopping to avoid overfitting of model

In [20]:
# fit the model
history = model.fit(
  train_x,
  train_y,
  validation_data=(val_x,val_y),
  epochs=10,
  callbacks=[early_stop],
  batch_size=32,shuffle=True)

Epoch 1/10
10/10 [==============================] - 52s 4s/step - loss: 8.9595 - accuracy: 0.1946 - val_loss: 7.8511 - val_accuracy: 0.2537
Epoch 2/10
10/10 [==============================] - 30s 3s/step - loss: 6.1491 - accuracy: 0.2315 - val_loss: 2.5272 - val_accuracy: 0.2985
Epoch 3/10
10/10 [==============================] - 30s 3s/step - loss: 3.1617 - accuracy: 0.3054 - val_loss: 2.1914 - val_accuracy: 0.4925
Epoch 4/10
10/10 [==============================] - 30s 3s/step - loss: 2.4574 - accuracy: 0.3087 - val_loss: 1.4623 - val_accuracy: 0.4030
Epoch 5/10
10/10 [==============================] - 30s 3s/step - loss: 2.0171 - accuracy: 0.3758 - val_loss: 1.4901 - val_accuracy: 0.3582
Epoch 6/10
10/10 [==============================] - 30s 3s/step - loss: 1.8379 - accuracy: 0.3557 - val_loss: 1.4843 - val_accuracy: 0.4030
Epoch 7/10
10/10 [==============================] - 31s 3s/step - loss: 1.5822 - accuracy: 0.3658 - val_loss: 1.1961 - val_accuracy: 0.4627
Epoch 8/10
10/10 [==

In [21]:
model.evaluate(test_x,test_y)

3/3 [==============================] - 6s 2s/step - loss: 1.0014 - accuracy: 0.6522


[1.0014019012451172, 0.6521739363670349]

In [22]:
predict = model.predict(test_x)

In [23]:
ans = [np.argmax(elements) for elements in predict]

In [25]:
model.save("resNet50Model.h5")

In [ ]:
plt.plot(history.history['accuracy'], label='train acc')

plt.plot(history.history['val_accuracy'], label='val acc')

plt.legend()

plt.savefig('vgg-acc-rps-1.png')

plt.show()

In [ ]:
# loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.savefig('vgg-loss-rps-1.png')
plt.show()